In [ ]:
import os
import time
import copy
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm

from utils.loss_function import SaliencyLoss
from utils.data_process import MyDataset

flag = 1 # 0 for TranSalNet_Dense, 1 for TranSalNet_Res

if flag:
    from TranSalNet_Res import TranSalNet
else:
    from TranSalNet_Dense import TranSalNet


↑↑↑ Set flag=1 to load TranSalNet_Dense,set flag=0 to load TranSalNet_Res.

In [ ]:
train_ids = pd.read_csv(r'datasets/train_ids.csv')
val_ids = pd.read_csv(r'datasets/val_ids.csv')
print(train_ids.iloc[1])
print(val_ids.iloc[1])

dataset_sizes = {'train':len(train_ids),'val':len(val_ids)}
print(dataset_sizes)

↑↑↑Load image id from dataset

In [ ]:
batch_size = 4
train_set = MyDataset(ids=train_ids,
                           stimuli_dir=r'datasets\train\train_stimuli/',
                           saliency_dir=r'datasets\train\train_saliency/',
                           fixation_dir=r'datasets\train\train_fixation/',
                           transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ]))

val_set = MyDataset(ids=val_ids,
                        stimuli_dir=r'datasets\val\val_stimuli/',
                        saliency_dir = r'datasets\val\val_saliency/',
                         fixation_dir=r'datasets\val\val_fixation/',
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ]))



dataloaders = {'train':DataLoader(train_set, batch_size=batch_size,shuffle=True, num_workers=4)
               ,'val':DataLoader(val_set, batch_size=batch_size,shuffle=False, num_workers=4)}


↑↑↑Set batch_size and Load dataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = TranSalNet()
model = model.to(device)

# Train the model below

In [ ]:

optimizer = optim.Adam(model.parameters(),lr=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

loss_fn = SaliencyLoss()

'''Training'''
best_model_wts = copy.deepcopy(model.state_dict())
num_epochs =30
best_loss = 100
for k,v in model.named_parameters():
    print('{}: {}'.format(k, v.requires_grad))

                 
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        
        
        # Iterate over data.
        for i_batch, sample_batched in tqdm(enumerate(dataloaders[phase])):
            stimuli, smap, fmap = sample_batched['image'], sample_batched['saliency'], sample_batched['fixation']
            stimuli, smap, fmap = stimuli.type(torch.cuda.FloatTensor), smap.type(torch.cuda.FloatTensor), fmap.type(torch.cuda.FloatTensor)
            stimuli, smap, fmap = stimuli.to(device), smap.to(device), fmap.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(stimuli)
                


                loss = -2*loss_fn(outputs,smap,loss_type='cc')\
                        -1*loss_fn(outputs,smap,loss_type='sim')+\
                        10*loss_fn(outputs,smap,loss_type='kldiv')-1*loss_fn(outputs,fmap,loss_type='nss')



                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * stimuli.size(0)



        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        
            
        print('{} Loss: {:.4f}'.format(
            phase, epoch_loss))
        
        
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            counter = 0
        elif phase == 'val' and epoch_loss >= best_loss:
            counter += 1
            if counter ==5:
                print('early stop!')
                break
    else:
        continue
    break
    print()


print('Best val loss: {:4f}'.format(best_loss))
model.load_state_dict(best_model_wts)

# Save the model below

In [ ]:
savepath = r'mymodel.pth'
torch.save(model.state_dict(),savepath)